In [ ]:
!pip install bentoml

In [ ]:
!pip install apache-flink

In [ ]:
!pip install transformers

In [ ]:
!pip install numpy==1.23.0

In [3]:
import bentoml
import transformers

pipe = transformers.pipeline("text-classification")

bentoml.transformers.save_model(
  "text-classification-pipe",
  pipe,
  signatures={
    "__call__": {"batchable": True}  # Enable dynamic batching for model
  }
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Model(tag="text-classification-pipe:gx6mbutubch3casc", path="/root/bentoml/models/text-classification-pipe/gx6mbutubch3casc/")

In [13]:
!python3 -V

Python 3.10.12


In [5]:
!bentoml models list


 Tag                                        Module                Size        Creation Time       
 text-classification-pipe:gx6mbutubch3casc  bentoml.transformers  256.35 MiB  2023-10-26 14:01:50 


In [9]:
import bentoml
import logging
import sys

from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.functions import RuntimeContext, MapFunction


class ClassifyFunction(MapFunction):
    def open(self, runtime_context: RuntimeContext):
        self.runner = bentoml.transformers.get("text-classification-pipe:gx6mbutubch3casc").to_runner()
        self.runner.init_local()

    def map(self, data):
        # transform(data)
        return data[0], self.runner.run(data[1])


def classify_tweets():
    # Create a StreamExecutionEnvironment
    env = StreamExecutionEnvironment.get_execution_environment()
    env.set_parallelism(1)

    # Create source DataStream, e.g. Kafka, Table, etc. Example uses data collection for simplicity.
    ds = env.from_collection(
        collection=[
            (1, "BentoML: Create an ML Powered Prediction Service in Minutes via @TDataScience https://buff.ly/3srhTw9 #Python #MachineLearning #BentoML"),
            (2, "Top MLOps Serving frameworks — 2021 https://link.medium.com/5Elq6Aw52ib #mlops #TritonInferenceServer #opensource #nvidia #machincelearning  #serving #tensorflow #PyTorch #Bodywork #BentoML #KFServing #kubeflow #Cortex #Seldon #Sagify #Syndicai"),
            (3, "#MLFlow provides components for experimentation management, ML project management. #BentoML only focuses on serving and deploying trained models"),
            (4, "2000 and beyond #OpenSource #bentoml"),
            (5, "Model Serving Made Easy https://github.com/bentoml/BentoML ⭐ 1.1K #Python #Bentoml #BentoML #Modelserving #Modeldeployment #Modelmanagement #Mlplatform #Mlinfrastructure #Ml #Ai #Machinelearning #Awssagemaker #Awslambda #Azureml #Mlops #Aiops #Machinelearningoperations #Turn")
        ]
    )

    # Define the execution logic
    ds = ds.map(ClassifyFunction())

    # Create sink and emit result to sink, e.g. Kafka, File, Table, etc. Example prints to stdout for simplicity.
    ds.print()

    # Submit for execution
    env.execute()


if __name__ == '__main__':
    logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(message)s")
    classify_tweets()